In [1]:
import json
import csv
import re
import sys
import pandas as pd
import numpy as np
import heapq

import gensim
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

#####################################################
# Functions
def csv_loader(PATH):
    text = pd.read_csv(PATH, names=['review','sentiment']) 
    return text

#####################################################
#args = sys.argv
#if len(args) < 2:
#    print("You forgot something")
FILE_1 = 'music_dev'  # name of interim csv file. For example: # games_train
FILE_2 = 'sew_val'  # name of comparison interim csv file. For example: # sew_train
N_DIS = 2   # number of dissimilar embeddings to select
# python3 cosine.py 'music_dev' 'sew_val' 2

In [2]:
# Load Interim CSV file and split into X and y
data_1 = csv_loader('../data/interim/' + FILE_1 + '.csv')
X_1, y_1 = data_1[['review']], data_1[['sentiment']]

# Load Interim CSV file and split into X and y
data_2 = csv_loader('../data/interim/' + FILE_2 + '.csv')
X_2, y_2 = data_2[['review']], data_2[['sentiment']]

In [20]:
# 
corp_1 = []
for i in range(5): 
    row = X_1.iloc[i]['review']
    corp_1.append(row)
    
# 
corp_2 = []
for i in range(3): 
    row = X_2.iloc[i]['review']
    corp_2.append(row)
corp_2 = corp_2[1:]
corp_2

['Four Stars Design is a bit off center',
 'Very Pleased with the Set This is the third time I have purchased these paints. The first time I bought a set of 12 and then purchased that same set again so I would not run out. Be aware that 12ml is a small qty. so these set work very nicely for a single project or in my case for a few small projects. I am using the set of 24 to paint designs on cedar feather boxes. The ranges of colors is this set offers much more artistic choice.']

In [21]:
count_vectorizer = TfidfVectorizer(stop_words='english')

TfidfVectorizer(stop_words='english')

In [23]:
sparse_matrix_1 = count_vectorizer.fit_transform(corp_1)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix_1 = sparse_matrix_1.todense()
df_1 = pd.DataFrame(doc_term_matrix_1, 
                  columns=count_vectorizer.get_feature_names_out())
df_1

,abe,acquainted,address,amazing,asleep,awesome,bc,big,boring,bought,...,unfortunately,unintelligible,uplifting,vinyl,visits,won,worked,world,worst,young
0,0.000000,0.000000,0.000000,0.000000,0.327465,0.000000,0.000000,0.000000,0.000000,0.132098,...,0.000000,0.000000,0.000000,0.000000,0.163733,0.000000,0.163733,0.000000,0.000000,0.000000
1,0.000000,0.188982,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188982,0.000000,...,0.188982,0.188982,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188982
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.183646,0.000000,0.183646,0.000000,0.183646,0.000000,0.000000,...,0.000000,0.000000,0.183646,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.155691,0.000000,0.311382,0.000000,0.000000,0.000000,0.155691,0.000000,0.000000,0.125611,...,0.000000,0.000000,0.000000,0.155691,0.000000,0.155691,0.000000,0.155691,0.155691,0.000000


In [14]:
sparse_matrix_2 = count_vectorizer.fit_transform(corp_2)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix_2 = sparse_matrix_2.todense()
df_2 = pd.DataFrame(doc_term_matrix_2, 
                  columns=count_vectorizer.get_feature_names_out())
df_2

,12,12ml,24,artistic,aware,bit,bought,boxes,case,cedar,...,qty,ranges,run,set,single,small,stars,time,using,work
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000
1,0.116248,0.116248,0.116248,0.116248,0.116248,0.0,0.116248,0.116248,0.116248,0.116248,...,0.116248,0.116248,0.116248,0.697486,0.116248,0.232495,0.0,0.232495,0.116248,0.116248


In [16]:
# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df_1, df_2))

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 108 while Y.shape[1] == 34